In [2]:
import codecs
import json
import pandas as pd
import json
import codecs
import re
import random
import numpy as np
import nltk
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.base import BaseEstimator, TransformerMixin

def read_dataset(path):
  with codecs.open(path, 'r', 'utf-8') as myFile:
    content = myFile.read()
  dataset = json.loads(content)
  return dataset

In [3]:
###load JSON file
file_name = 'data/pizza_request_dataset.json'
dataset = read_dataset(file_name)
df = pd.read_json(json.dumps(dataset, sort_keys=True, indent=2))

###create random 90-10 split
X = df
print len(X)

rows = random.sample(X.index, int(0.9*len(X)) + 1)
X_train = X.ix[rows]
X_test = X.drop(rows)
y_train = X_train.requester_received_pizza.astype(int)
y_test = X_test.requester_received_pizza.astype(int)

print "Data loading and train-test splits = DONE!"
###

###subsample data
# print "Subsampling"
# subsample = 30
# X_train, y_train, X_test, y_test = X_train[:subsample], y_train[:subsample], X_test[:subsample], y_test[:subsample]

5671
Data loading and train-test splits = DONE!


In [4]:
print "part 1"

part 1


In [5]:
###Model 1 - a) n-grams

### build pipeline; fit train; predict on test
import nltk
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB

##vectorizer arguments blah!

tokenizer=None#word_tokenize
# stop_words=nltk.corpus.stopwords.words("english")#None
ngram_range=(1, 2)
lowercase=True
max_features=500
binary=False
dtype=np.float64

class TextExtractor(BaseEstimator, TransformerMixin):

    def __init__(self, vars):
        self.vars = vars  # e.g. pass in a column name to extract

    def transform(self, X, y=None):
        return X[self.vars]  # where the actual feature extraction happens

    def fit(self, X, y=None):
        return self  # generally does nothing

###create unigram vectorizer
uniVect = CountVectorizer(decode_error="ignore",
#                                tokenizer=tokenizer,
#                                stop_words=stop_words,
                               ngram_range=(1,1),
                               lowercase=lowercase,
                               binary=binary,
                               dtype=dtype,
                               max_features=max_features)

###create bigram vect
biVect = CountVectorizer(decode_error="ignore",
#                                tokenizer=tokenizer,
#                                stop_words=stop_words,
                               ngram_range=(2,2),
                               lowercase=lowercase,
                               binary=binary,
                               dtype=dtype,
                               max_features=max_features)

# load custom features and FeatureUnion with Vectorizer
features = []
features.append(('unigram', uniVect))
features.append(('bigram', biVect))
all_features = FeatureUnion(features)

linear_svc = svm.SVC(kernel='linear', probability=True)

###create pipeline
text_clf = Pipeline([
                     ('getText', TextExtractor('request_text_edit_aware')),
                     ('all', all_features),
#                      ('tfidf', TfidfTransformer()),
#                     ('clf', linear_svc),
                    ('clf', MultinomialNB(alpha=0.1)),

                    ])

# text_clf = Pipeline([
#                      ('vect', vectorizer),
# #                      ('tfidf', TfidfTransformer()),
#                     ('clf', linear_svc),
#                     ])

###fit training data
print "Fitting part 1!"
text_clf = text_clf.fit(X_train, y_train)

### predict on test data
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)
# print "NO!"
probas = text_clf.predict_proba(X_test)
# print "YO!"
###get performance metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy = accuracy_score(y_test,predicted)
#         accuracy = np.mean(y_CVtest == predicted)
precision, recall, fscore, sup = precision_recall_fscore_support(y_test, predicted, average='binary', pos_label=0)
print precision, recall, fscore, accuracy# np.mean(predicted == y_test)
from sklearn.metrics import roc_auc_score
# roc_auc = roc_auc_score(y_test, predicted)
roc_auc = roc_auc_score(y_test, probas[:,1])
print "ROC = ", roc_auc

from sklearn.metrics import confusion_matrix
C = confusion_matrix(y_test, predicted)
#true negatives C_{0,0}, false negatives is C_{1,0}, true positives is C_{1,1} and false positives is C_{0,1}
tn = C[0][0]
fn = C[1][0]
tp = C[1][1]
fp = C[0][1]
##since labels are switched, invert the confusion matrix outputs: Pos = 0, Neg = 1
#specificity = TN/TN+FP = tp/tp+fn
 
print "Specificity: ", float(tp)/(tp+fn)
# print "Precision: ", float(tn)/(tn+fn)
# print "Recall: ", float(tn)/(tn+fp)
# print "Accuracy: ", float(tn+tp)/(tn+tp+fp+fn)

Fitting part 1!
0.804395604396 0.847222222222 0.825253664036 0.726631393298
ROC =  0.621416323731
Specificity:  0.340740740741


In [6]:
print "part 2"
# 0.8125 0.854875283447 0.833149171271 0.733686067019
# ROC =  0.563033149768

part 2


In [7]:
###Model 2 - a) custom features

### build pipeline; fit train; predict on test
import nltk
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.preprocessing.data import StandardScaler, MinMaxScaler


class ColumnExtractor(BaseEstimator, TransformerMixin):

    def __init__(self, vars):
        self.vars = vars  # e.g. pass in a column name to extract

    def transform(self, X, y=None):
        return X[self.vars]  # where the actual feature extraction happens

    def fit(self, X, y=None):
        return self  # generally does nothing
    
class StringExtractor(BaseEstimator, TransformerMixin):

    def __init__(self, vars):
        self.vars = vars  # e.g. pass in a column name to extract

    def transform(self, X, y=None):
        return X[self.vars].astype(str)  # where the actual feature extraction happens

    def fit(self, X, y=None):
        return self  # generally does nothing
    
class SubredditExtractor(BaseEstimator, TransformerMixin):

    def __init__(self, vars):
        self.vars = vars  # e.g. pass in a column name to extract

    def transform(self, X, y=None):
        return X[self.vars].apply(lambda x: ' '.join(x))  # where the actual feature extraction happens

    def fit(self, X, y=None):
        return self  # generally does nothing

##vectorizer arguments blah!
tokenizer=None#word_tokenize
# stop_words=nltk.corpus.stopwords.words("english")#None
ngram_range=(1, 1)
lowercase=True
max_features=10000
binary=False
dtype=np.float64
###create unigram vectorizer
subVect = CountVectorizer(decode_error="ignore",
#                                tokenizer=tokenizer,
#                                stop_words=stop_words,
                               ngram_range=ngram_range,
                               lowercase=lowercase,
                               binary=binary,
                               dtype=dtype,
                               max_features=max_features)

flairVect = CountVectorizer(decode_error="ignore",
#                                tokenizer=tokenizer,
#                                stop_words=stop_words,
                               ngram_range=ngram_range,
                               lowercase=lowercase,
                               binary=binary,
                               dtype=dtype,
                               max_features=100)

# load custom features for FeatureUnion
cols_act = [
'post_was_edited',
'requester_account_age_in_days_at_request',
'requester_account_age_in_days_at_retrieval',
'requester_days_since_first_post_on_raop_at_request',
'requester_days_since_first_post_on_raop_at_retrieval',
'requester_number_of_comments_at_request',
'requester_number_of_comments_at_retrieval',
'requester_number_of_comments_in_raop_at_request',
'requester_number_of_comments_in_raop_at_retrieval',
'requester_number_of_posts_at_request',
'requester_number_of_posts_at_retrieval',
'requester_number_of_posts_on_raop_at_request',
'requester_number_of_posts_on_raop_at_retrieval',
'requester_number_of_subreddits_at_request',
# 'requester_subreddits_at_request',
]

cols_rep = [
'number_of_downvotes_of_request_at_retrieval',
'number_of_upvotes_of_request_at_retrieval',
'requester_upvotes_minus_downvotes_at_request', ###this contains negative values
'requester_upvotes_minus_downvotes_at_retrieval',###this contains negative values
'requester_upvotes_plus_downvotes_at_request',
'requester_upvotes_plus_downvotes_at_retrieval',
### 'requester_user_flair',
]

get_flair = Pipeline([
                     ('getFlair', StringExtractor('requester_user_flair')),
                     ('counts', flairVect),
                    ])

get_subs = Pipeline([
                     ('getSubs', SubredditExtractor('requester_subreddits_at_request')),
                     ('counts', subVect),
                    ])
##feature union
features = []
features.append(('activity', ColumnExtractor(cols_act)))
features.append(('reputation', ColumnExtractor(cols_rep) ))
features.append(('flair', get_flair))
features.append(('subs', get_subs))

all_features = FeatureUnion(features)

##scaling the features to make it stop executing bruh!
source_scaler = StandardScaler(copy=False, with_mean=False, with_std=True)
# source_scaler = MinMaxScaler()

linear_svc = svm.SVC(kernel='linear', probability=True)

###create pipeline
text_clf = Pipeline([
#                      ('getText', TextExtractor('request_text_edit_aware')),
                     ('all', all_features),
                    ('scaler', source_scaler),
#                      ('tfidf', TfidfTransformer()),
#                     ('clf', LinearSVC(C=0.1)),
#                     ('clf', MultinomialNB(alpha=0.1)),
                    ('clf', linear_svc),
                    ])

###fit training data
print "Fitting part 2!"
text_clf = text_clf.fit(X_train, y_train)

### predict on test data
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)
# print "NO!"
# probas = text_clf.predict_proba(X_test)
# print "YO!"
###get performance metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy = accuracy_score(y_test,predicted)
#         accuracy = np.mean(y_CVtest == predicted)
precision, recall, fscore, sup = precision_recall_fscore_support(y_test, predicted, average='binary', pos_label=0)
print precision, recall, fscore, accuracy# np.mean(predicted == y_test)
# from sklearn.metrics import roc_auc_score
#### roc_auc = roc_auc_score(y_test, predicted)
roc_auc = roc_auc_score(y_test, probas[:,1])
print "ROC = ", roc_auc

from sklearn.metrics import confusion_matrix
C = confusion_matrix(y_test, predicted)
#true negatives C_{0,0}, false negatives is C_{1,0}, true positives is C_{1,1} and false positives is C_{0,1}
tn = C[0][0]
fn = C[1][0]
tp = C[1][1]
fp = C[0][1]
##since labels are switched, invert the confusion matrix outputs: Pos = 0, Neg = 1
#specificity = TN/TN+FP = tp/tp+fn
 
print "Specificity: ", float(tp)/(tp+fn)
# print "Precision: ", float(tn)/(tn+fn)
# print "Recall: ", float(tn)/(tn+fp)
# print "Accuracy: ", float(tn+tp)/(tn+tp+fp+fn)

Fitting part 2!
0.995316159251 0.983796296296 0.989522700815 0.984126984127
ROC =  0.621416323731
Specificity:  0.985185185185


In [104]:
from sklearn.metrics import confusion_matrix
C = confusion_matrix(y_test, predicted)
#true negatives C_{0,0}, false negatives is C_{1,0}, true positives is C_{1,1} and false positives is C_{0,1}
tn = C[0][0]
fn = C[1][0]
tp = C[1][1]
fp = C[0][1]
##since labels are switched, invert the confusion matrix outputs: Pos = 0, Neg = 1
#specificity = TN/TN+FP = tp/tp+fn
 
print "Specificity: ", float(tp)/(tp+fn)
# print "Precision: ", float(tn)/(tn+fn)
# print "Recall: ", float(tn)/(tn+fp)
# print "Accuracy: ", float(tn+tp)/(tn+tp+fp+fn)

Specificity:  1.0


In [92]:
flairVect.vocabulary_
len(subVect.vocabulary_)

8423

In [8]:
print 'part 3: Narratives'
print "Running in 4687.pts-33.flashmob03"

part 3: Narratives
Running in 4687.pts-33.flashmob03


In [16]:
##vectorizer arguments blah!
tokenizer=None#word_tokenize
# stop_words=nltk.corpus.stopwords.words("english")#None
ngram_range=(1, 1)
lowercase=True
max_features=None
binary=False
dtype=np.float64

desire_vocab = pd.read_csv('resources/narratives/desire.txt', names = ["words"])
###create unigram vectorizer for desire features
desireVect = CountVectorizer(
                               vocabulary = desire_vocab.words,
                               decode_error="ignore",
#                                tokenizer=tokenizer,
#                                stop_words=stop_words,
                               ngram_range=(1,1),
                               lowercase=lowercase,
#                                binary=binary,
                               dtype=dtype,
#                                max_features=max_features
                            )

family_vocab = pd.read_csv('resources/narratives/family.txt', names = ["words"])
###create unigram vectorizer for family_vocab features
familyVect = CountVectorizer(
                               vocabulary = family_vocab.words,
                               decode_error="ignore",
#                                tokenizer=tokenizer,
#                                stop_words=stop_words,
                               ngram_range=(1,1),
                               lowercase=lowercase,
#                                binary=binary,
                               dtype=dtype,
#                                max_features=max_features
                            )

job_vocab = pd.read_csv('resources/narratives/job.txt', names = ["words"])
###create unigram vectorizer for job_vocab features
jobVect = CountVectorizer(
                               vocabulary = job_vocab.words,
                               decode_error="ignore",
#                                tokenizer=tokenizer,
#                                stop_words=stop_words,
                               ngram_range=(1,1),
                               lowercase=lowercase,
#                                binary=binary,
                               dtype=dtype,
#                                max_features=max_features
                            )

money_vocab = pd.read_csv('resources/narratives/money.txt', names = ["words"])
###create unigram vectorizer for money_vocab features
###note that this list contains duplicate terms: bills, due
moneyVect = CountVectorizer(
                               vocabulary = money_vocab.words,
                               decode_error="ignore",
#                                tokenizer=tokenizer,
#                                stop_words=stop_words,
                               ngram_range=(1,1),
                               lowercase=lowercase,
#                                binary=binary,
                               dtype=dtype,
#                                max_features=max_features
                            )

student_vocab = pd.read_csv('resources/narratives/student.txt', names = ["words"])
###create unigram vectorizer for student features
studentVect = CountVectorizer(
                               vocabulary = student_vocab.words,
                               decode_error="ignore",
#                                tokenizer=tokenizer,
#                                stop_words=stop_words,
                               ngram_range=(1,1),
                               lowercase=lowercase,
#                                binary=binary,
                               dtype=dtype,
#                                max_features=max_features
                            )


##feature union
features = []
features.append(('desire', desireVect))
features.append(('family', familyVect))
features.append(('job', jobVect))
features.append(('money', moneyVect))
features.append(('student', studentVect))
all_features = FeatureUnion(features)

##scaling the features to make it stop executing bruh!
source_scaler = StandardScaler(copy=False, with_mean=False, with_std=True)
# source_scaler = MinMaxScaler()
# MinMaxScaler

linear_svc = svm.SVC(kernel='linear', probability=True)

###create pipeline
text_clf = Pipeline([
                     ('getText', TextExtractor('request_text_edit_aware')),
                     ('all', all_features),
                     ('scaler', source_scaler),
#                      ('tfidf', TfidfTransformer()),
                    ('clf', MultinomialNB(alpha=0.1)),
#                     ('clf', linear_svc),
                    ])

###fit training data
print "Fitting part 3!"
text_clf = text_clf.fit(X_train, y_train)

### predict on test data
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)
# print "NO!"
probas = text_clf.predict_proba(X_test)
# print "YO!"
###get performance metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy = accuracy_score(y_test,predicted)
#         accuracy = np.mean(y_CVtest == predicted)
precision, recall, fscore, sup = precision_recall_fscore_support(y_test, predicted, average='binary', pos_label=0)
print precision, recall, fscore, accuracy# np.mean(predicted == y_test)
from sklearn.metrics import roc_auc_score
# roc_auc = roc_auc_score(y_test, predicted)
roc_auc = roc_auc_score(y_test, probas[:,1])
print "ROC = ", roc_auc
from sklearn.metrics import confusion_matrix
C = confusion_matrix(y_test, predicted)
#true negatives C_{0,0}, false negatives is C_{1,0}, true positives is C_{1,1} and false positives is C_{0,1}
tn = C[0][0]
fn = C[1][0]
tp = C[1][1]
fp = C[0][1]
##since labels are switched, invert the confusion matrix outputs: Pos = 0, Neg = 1
#specificity = TN/TN+FP = tp/tp+fn
 
print "Specificity: ", float(tp)/(tp+fn)
# print "Precision: ", float(tn)/(tn+fn)
# print "Recall: ", float(tn)/(tn+fp)
# print "Accuracy: ", float(tn+tp)/(tn+tp+fp+fn)

Fitting part 3!
0.797727272727 0.8125 0.80504587156 0.700176366843
ROC =  0.572813786008
Specificity:  0.340740740741


In [17]:
print "part 4: Moral foundations"
predicted

part 4: Moral foundations


array([0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0,

In [18]:
###parse the dictionary bro
with open('resources/MoralFoundations.dic', 'r') as f:
    lines = f.readlines()
count = 0
l = 0

##tokenizer to get the number of words in the sentence
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
##
# list = [][]
list_1 = []
list_2 = []
list_3 = []
list_4 = []
list_5 = []
list_6 = []
list_7 = []
list_8 = []
list_9 = []
list_10 = []
list_11 = []

print "Parsing Moral Foundations dict"
for line in lines:
    if "%" in line:
        count +=1 
#         print line
        continue
    if count < 2:
        continue
    first = 0
    for word in tokenizer.tokenize(line):
#         print word
        if first == 0:
            if "*" in line:
                word = word + ".*"
            term = word
#             print "Word: ", term
        
        if first > 0:
            label = int(word)
#             print "Label: ", label
            if label == 1:
                list_1.append(term)
            elif label == 2:
                list_2.append(term)
            elif label == 3:
                list_3.append(term)
            elif label == 4:
                list_4.append(term)
            elif label == 5:
                list_5.append(term)
            elif label == 6:
                list_6.append(term)
            elif label == 7:
                list_7.append(term)
            elif label == 8:
                list_8.append(term)
            elif label == 9:
                list_9.append(term)
            elif label == 10:
                list_10.append(term)
            elif label == 11:
                list_11.append(term)
        first += 1
#     print "---"
    l+= 1
# print "Lines done = ", l
moral_foundations = []
moral_foundations.append(list_1)
moral_foundations.append(list_2)
moral_foundations.append(list_3)
moral_foundations.append(list_4)
moral_foundations.append(list_5)
moral_foundations.append(list_6)
moral_foundations.append(list_7)
moral_foundations.append(list_8)
moral_foundations.append(list_9)
moral_foundations.append(list_10)
moral_foundations.append(list_11)    

Parsing Moral Foundations dict


In [76]:
# moral_foundations[10]
# postbanhate_iter += np.sum(arr)
# for text in df1.body:
#     prebanWords_iter += len(tokenizer.tokenize(text))

In [86]:
# ptrn = r"\b("+ "|".join(moral_foundations[0]) + r")\b"
import functools
from functools import partial

##tokenizer to get the number of words in the sentence
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

def apply_func(func, X, *args, **kwargs):
    '''
    Apply a function that applies to a single row to an entire matrix 
    
    :param func: the function
    :param X: the matrix
    
    :returns: a new matrix, where each row is the func applied to the rows of X
    '''
    func_ = partial(func, *args, **kwargs)
    X_ = np.array([func_(i) for i in X])
    return X_.reshape((len(X), np.prod(X_.shape)/len(X)))


def get_scores(text, mf_label):
    ###count number of words in the sentence
    RE = re.compile('[0-9a-z-]', re.I)
    words = filter(lambda w: RE.search(w) and w.replace('-', ''), tokenizer.tokenize(text))
    wordc = max(1, len(words))
    print wordc
    
    ###count number of matches with MF dict entries
    ptrn = r"\b("+ "|".join(moral_foundations[mf_label]) + r")\b"
    str_ = "safer safe safest something!"
    reg = re.compile(ptrn)
    score = 0
    for word in tokenizer.tokenize(str_):
        score += len(reg.findall(word))

    return float(score)/wordc

class TextExtractor(BaseEstimator, TransformerMixin):

    def __init__(self, vars):
        self.vars = vars  # e.g. pass in a column name to extract

    def transform(self, X, y=None):
        return X[self.vars]  # where the actual feature extraction happens

    def fit(self, X, y=None):
        return self  # generally does nothing

class FunctionTransformer(TransformerMixin, BaseEstimator):
    def __init__(self, the_function):
        '''
        Wrap a function into the transform method of a transformer.
        
        Use: 
            def plus_one(X):
                return X+1
            
            f = FunctionTransformer(plus_one)
            f.fit_transform(np.array([[1, 2, 3]]), [0, 0, 1])
            Out[21]: array([[2, 3, 4]])        
            
        :param the_function: function that takes the matrix X as the first argument, 
            and outputs the desired transformed X 
        '''
        self.the_function = the_function
    def fit(self, *args, **kwargs):#does nothing at all
        return self
    def transform(self, X, **transform_params):
        return self.the_function(X)



##feature union
features = []

# for i in range(0,11):
#     print i
getMFTransformer = FunctionTransformer(partial(apply_func, get_scores, mf_label=0))
#     features.append(('mf'+i.astype(str), FunctionTransformer(partial(apply_func, get_scores, mf_label=i))))
all_features = FeatureUnion(features)
# print len(all_features)
##scaling the features to make it stop executing bruh!
source_scaler = StandardScaler(copy=False, with_mean=False, with_std=True)
# source_scaler = MinMaxScaler()
# MinMaxScaler

linear_svc = svm.SVC(kernel='linear', probability=True)

###create pipeline
text_clf = Pipeline([
                     ('getText', TextExtractor('request_text_edit_aware')),
                     ('all', all_features),
                     ('scaler', source_scaler),
#                      ('tfidf', TfidfTransformer()),
#                     ('clf', MultinomialNB(alpha=0.1)),
                    ('clf', linear_svc),
                    ])

###fit training data
print "Fitting part 3!"
text_clf = text_clf.fit(X_train, y_train)

### predict on test data
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)
# print "NO!"
probas = text_clf.predict_proba(X_test)
# print "YO!"
###get performance metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy = accuracy_score(y_test,predicted)
#         accuracy = np.mean(y_CVtest == predicted)
precision, recall, fscore, sup = precision_recall_fscore_support(y_test, predicted, average='binary', pos_label=0)
print precision, recall, fscore, accuracy# np.mean(predicted == y_test)
from sklearn.metrics import roc_auc_score
# roc_auc = roc_auc_score(y_test, predicted)
roc_auc = roc_auc_score(y_test, probas[:,1])
print "ROC = ", roc_auc
from sklearn.metrics import confusion_matrix
C = confusion_matrix(y_test, predicted)
#true negatives C_{0,0}, false negatives is C_{1,0}, true positives is C_{1,1} and false positives is C_{0,1}
tn = C[0][0]
fn = C[1][0]
tp = C[1][1]
fp = C[0][1]
##since labels are switched, invert the confusion matrix outputs: Pos = 0, Neg = 1
#specificity = TN/TN+FP = tp/tp+fn
 
print "Specificity: ", float(tp)/(tp+fn)
# print "Precision: ", float(tn)/(tn+fn)
# print "Recall: ", float(tn)/(tn+fp)
# print "Accuracy: ", float(tn+tp)/(tn+tp+fp+fn)

Fitting part 3!


ValueError: need more than 0 values to unpack

In [75]:
ptrn = r"\b("+ "|".join(moral_foundations[10]) + r")\b"
str_ = "safelexs ethical safe safest something!"

reg = re.compile(ptrn)
score = 0

for word in tokenizer.tokenize(str_):
    score += len(reg.findall(word))

print score
#     print len(reg.match(str_).group())
# if re.compile(ptrn).match(str_).group() == None:
#     print "No match"

0


In [20]:
def getCounts(text):
    string = ""
    ptrn = r"\b("+ "|".join(moral_foundations[0]) + r")\b"
    

In [84]:
i.to_String

AttributeError: 'int' object has no attribute 'to_String'

In [57]:
for text in X_train['request_text_edit_aware']:
    RE = re.compile('[0-9a-z-]', re.I)
    words = filter(lambda w: RE.search(w) and w.replace('-', ''), tokenizer.tokenize(text))
    wordc = max(1, len(words))
    print wordc
    if wordc <= 10:
        print text

102
25
69
33
72
45
229
126
1
 
112
81
27
36
156
1

2
edit: my*
68
7
Biochemistry is a hell of a major. 
47
185
66
59
81
46
44
17
138
58
147
45
55
35
117
66
58
61
1

177
52
130
34
38
82
101
23
1

89
15
4
What a sad time.
70
7
AHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH!!!!!!!!!

[VA area, serious offers only please]
103
73
44
118
82
72
57
59
46
66
57
16
57
83
69
11
39
46
74
19
122
23
24
12
116
113
101
131
42
95
86
179
56
1

101
9
*just to be clear: by CA, I mean California
41
30
12
54
112
57
63
63
48
96
59
37
44
32
25
73
158
39
55
98
137
117
21
127
53
13
109
114
43
160
33
55
97
1

35
203
23
8
Plus, I've never met any local redditors.
72
125
113
63
31
85
66
1

43
95
35
42
1

64
12
68
85
63
39
121
102
54
1
Fail
76
68
34
23
63
69
78
49
108
296
83
50
27
58
130
132
31
62
58
60
95
21
18
224
26
38
50
5
Thank you for your consideration. :)
172
35
100
84
133
76
102
99
31
79
147
77
103
90
87
5
It would be much appreciated.
58
40
16
127
32
40
177
57
27
34
45
118
71
60
59
62
28
137
44
29
47
86
24
68
7
No s

In [ ]:
###MF pipeline
# ptrn = r"\b("+ "|".join(moral_foundations[0]) + r")\b"
import functools
from functools import partial

##tokenizer to get the number of words in the sentence
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

def apply_func(func, X, *args, **kwargs):
    '''
    Apply a function that applies to a single row to an entire matrix 
    
    :param func: the function
    :param X: the matrix
    
    :returns: a new matrix, where each row is the func applied to the rows of X
    '''
    func_ = partial(func, *args, **kwargs)
    X_ = np.array([func_(i) for i in X])
    return X_.reshape((len(X), np.prod(X_.shape)/len(X)))


def get_scores(text, mf_label):
    ###count number of words in the sentence
    RE = re.compile('[0-9a-z-]', re.I)
    words = filter(lambda w: RE.search(w) and w.replace('-', ''), tokenizer.tokenize(text))
    wordc = max(1, len(words))
    print wordc
    
    ###count number of matches with MF dict entries
    ptrn = r"\b("+ "|".join(moral_foundations[mf_label]) + r")\b"
    str_ = "safer safe safest something!"
    reg = re.compile(ptrn)
    score = 0
    for word in tokenizer.tokenize(str_):
        score += len(reg.findall(word))

    return float(score)/wordc

class TextExtractor(BaseEstimator, TransformerMixin):

    def __init__(self, vars):
        self.vars = vars  # e.g. pass in a column name to extract

    def transform(self, X, y=None):
        return X[self.vars]  # where the actual feature extraction happens

    def fit(self, X, y=None):
        return self  # generally does nothing

class FunctionTransformer(TransformerMixin, BaseEstimator):
    def __init__(self, the_function):
        '''
        Wrap a function into the transform method of a transformer.
        
        Use: 
            def plus_one(X):
                return X+1
            
            f = FunctionTransformer(plus_one)
            f.fit_transform(np.array([[1, 2, 3]]), [0, 0, 1])
            Out[21]: array([[2, 3, 4]])        
            
        :param the_function: function that takes the matrix X as the first argument, 
            and outputs the desired transformed X 
        '''
        self.the_function = the_function
    def fit(self, *args, **kwargs):#does nothing at all
        return self
    def transform(self, X, **transform_params):
        return self.the_function(X)



##feature union
features = []
getMFTransformer1 = 
features.append(('mf', getMFTransformer))
all_features = FeatureUnion(features)

##scaling the features to make it stop executing bruh!
source_scaler = StandardScaler(copy=False, with_mean=False, with_std=True)
# source_scaler = MinMaxScaler()
# MinMaxScaler

linear_svc = svm.SVC(kernel='linear', probability=True)

###create pipeline
text_clf = Pipeline([
                     ('getText', TextExtractor('request_text_edit_aware')),
                     ('all', all_features),
                     ('scaler', source_scaler),
#                      ('tfidf', TfidfTransformer()),
#                     ('clf', MultinomialNB(alpha=0.1)),
                    ('clf', linear_svc),
                    ])

###fit training data
print "Fitting part 3!"
text_clf = text_clf.fit(X_train, y_train)

### predict on test data
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)
# print "NO!"
probas = text_clf.predict_proba(X_test)
# print "YO!"
###get performance metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
accuracy = accuracy_score(y_test,predicted)
#         accuracy = np.mean(y_CVtest == predicted)
precision, recall, fscore, sup = precision_recall_fscore_support(y_test, predicted, average='binary', pos_label=0)
print precision, recall, fscore, accuracy# np.mean(predicted == y_test)
from sklearn.metrics import roc_auc_score
# roc_auc = roc_auc_score(y_test, predicted)
roc_auc = roc_auc_score(y_test, probas[:,1])
print "ROC = ", roc_auc
from sklearn.metrics import confusion_matrix
C = confusion_matrix(y_test, predicted)
#true negatives C_{0,0}, false negatives is C_{1,0}, true positives is C_{1,1} and false positives is C_{0,1}
tn = C[0][0]
fn = C[1][0]
tp = C[1][1]
fp = C[0][1]
##since labels are switched, invert the confusion matrix outputs: Pos = 0, Neg = 1
#specificity = TN/TN+FP = tp/tp+fn
 
print "Specificity: ", float(tp)/(tp+fn)
# print "Precision: ", float(tn)/(tn+fn)
# print "Recall: ", float(tn)/(tn+fp)
# print "Accuracy: ", float(tn+tp)/(tn+tp+fp+fn)